# Use Flow as Component in Pipeline
[why we need use flow in AzureML pipeline]

[typical user scenarios]

[user benifits when use flow in pipeline]


**Requirements** - In order to benefit from this tutorial, you will need:
- A basic understanding of Machine Learning
- An Azure account with an active subscription - [Create an account for free](https://azure.microsoft.com/free/?WT.mc_id=A261C142F)
- An Azure ML workspace with computer cluster - [Configure workspace](../../configuration.ipynb)
- A python environment
- Installed Azure Machine Learning Python SDK v2 - [install instructions](../../../README.md) - check the getting started section

**Learning Objectives** - By the end of this tutorial, you should be able to:
- Connect to your AML workspace from the Python SDK
- Create `Pipeline` with a component loaded from `flow.dag.yaml`

**Motivations** - This notebook explains how to run a pipeline with distributed training component.

# 1. Connect to Azure Machine Learning Workspace

The [workspace](https://docs.microsoft.com/en-us/azure/machine-learning/concept-workspace) is the top-level resource for Azure Machine Learning, providing a centralized place to work with all the artifacts you create when you use Azure Machine Learning. In this section we will connect to the workspace in which the job will be run.

## 1.1 Import the required libraries

In [ ]:
# import required libraries
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential

from azure.ai.ml import MLClient, load_component, Input
from azure.ai.ml.constants import AssetTypes
from azure.ai.ml.dsl import pipeline

## 1.2 Configure credential

We are using `DefaultAzureCredential` to get access to workspace. 
`DefaultAzureCredential` should be capable of handling most Azure SDK authentication scenarios. 

Reference for more available credentials if it does not work for you: [configure credential example](../../configuration.ipynb), [azure-identity reference doc](https://docs.microsoft.com/en-us/python/api/azure-identity/azure.identity?view=azure-python).

In [ ]:
try:
    credential = DefaultAzureCredential()
    # Check if given credential can get token successfully.
    credential.get_token("https://management.azure.com/.default")
except Exception as ex:
    # Fall back to InteractiveBrowserCredential in case DefaultAzureCredential not work
    credential = InteractiveBrowserCredential()

## 1.3 Get a handle to the workspace

We use config file to connect to a workspace. The Azure ML workspace should be configured with computer cluster. [Check this notebook for configure a workspace](../../configuration.ipynb)

In [ ]:
# Get a handle to workspace
ml_client = MLClient.from_config(credential=credential)

# Retrieve an already attached Azure Machine Learning Compute.
cluster_name = "cpu-cluster"
print(ml_client.compute.get(cluster_name))

# 2. Load flow as component

We suppose that there has already been a flow authored with Promptflow SDK/CLI/portal. Then we can load its flow dag yaml as a component like regular component specs.

In [ ]:
flow_component = load_component("../../flows/standard/web-classification/flow.dag.yaml")

# 3. Pipeline job
## 3.1 Build pipeline

In [ ]:
data_input = Input(
    path="../../flows/standard/web-classification/data.jsonl", type=AssetTypes.URI_FILE
)


@pipeline()
def pipeline_func_with_flow(data):
    flow_node = flow_component(
        data=data,
        url="${data.url}",
        connections={
            "summarize_text_content": {
                "connection": "azure_open_ai_connection",
                "deployment_name": "gpt-35-turbo",
            },
            "classify_with_llm": {
                "connection": "azure_open_ai_connection",
                "deployment_name": "gpt-35-turbo",
            },
        },
    )
    flow_node.compute = "cpu-cluster"


# create pipeline instance
pipeline_job = pipeline_func_with_flow(data=data_input)

In [ ]:
data_input = Input(
    path="../../flows/standard/web-classification/data.jsonl", type=AssetTypes.URI_FILE
)

data_prep_component = load_component("./components/data-prep/data-prep.yaml")
result_parser_component = load_component("./components/result-parser/result-parser.yaml")
flow_component = load_component("../../flows/standard/web-classification/flow.dag.yaml")
@pipeline()
def pipeline_func_with_flow(data):
    data_prep_node = data_prep_component(
        input_data_file=data,
    )
    data_prep_node.compute = "cpu-cluster"

    flow_node = flow_component(
        data=data_prep_node.outputs.output_data_folder,
        url="${data.url}",
        connections={
            "summarize_text_content": {
                "connection": "azure_open_ai_connection",
                "deployment_name": "gpt-35-turbo",
            },
            "classify_with_llm": {
                "connection": "azure_open_ai_connection",
                "deployment_name": "gpt-35-turbo",
            },
        },
    )

    flow_node.environment_variables = {"PF_INPUT_FORMAT": "csv"}
    flow_node.compute = "cpu-cluster"
    flow_node.resources = { 'instance_count': 3 }
    flow_node.outputs.flow_outputs.mode = "rw_mount"

    result_parser_node = result_parser_component(
        source_data=data_prep_node.outputs.output_data_folder,
        pf_output_data=flow_node.outputs.flow_outputs,
    )
    
    result_parser_node.compute = "cpu-cluster"

# create pipeline instance
pipeline_job = pipeline_func_with_flow(data=data_input)

## 3.2 Submit pipeline job

In [9]:
# submit job to workspace
pipeline_job = ml_client.jobs.create_or_update(
    pipeline_job, experiment_name="pipeline_samples"
)
pipeline_job

Experiment,Name,Type,Status,Details Page
pipeline_samples,bubbly_room_jcsyk2wbts,pipeline,Preparing,Link to Azure Machine Learning studio


In [ ]:
# Wait until the job completes
ml_client.jobs.stream(pipeline_job.name)

# Next Steps
You can see further examples of running a pipeline job [here](../)

In [ ]:
# test section 
import pandas as pd

input_data = pd.read_csv("./processed_data.csv")
output_data = pd.read_json("./parallel_run_step.jsonl", lines=True)
print(output_data.head())

if len(input_data) != len(output_data):
    raise Exception("Index mismatch between data source and result")

input_data = input_data.merge(output_data, how='left', left_index=True, right_on="line_number", suffixes=('', '_predict'))

print(input_data.head())

with open("./processed_data_with_predictions.jsonl", "w") as file:
    file.write(input_data.to_json(orient="records", lines=True))

